In [1]:
import pandas as pd

In [2]:
# 分析单个nasdaq文件

# 指定文件路径
file_path = './nasdaq/csv/GOOD.csv'

# 读取 CSV 文件
df = pd.read_csv(file_path)

# 显示前几行数据
df.head()

,Date,Low,Open,Volume,High,Close,Adjusted Close
0,14-08-2003,14.65,15.15,1176700,15.15,14.90,3.218931
1,15-08-2003,14.95,15.00,259600,15.25,15.25,3.294542
2,18-08-2003,15.20,15.28,47900,15.28,15.25,3.294542
3,19-08-2003,15.10,15.25,64800,15.25,15.15,3.272936
4,20-08-2003,15.10,15.20,86300,15.34,15.21,3.285900


In [76]:
# 转换日期为 datetime 格式
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

# 定义事件 A: Close 值下降, 事件 B: Close 值上升
df['Close_Change'] = df['Close'].diff()

# 初始化状态机识别
pattern_start = []
pattern_end = []
in_pattern = False

for i in range(1, len(df)):
    # 识别事件A: Close值下降
    if df['Close_Change'].iloc[i] < 0:
        if not in_pattern:
            pattern_start.append(i)
        in_pattern = True
    # 识别事件B: Close值上升，结束模式匹配
    elif df['Close_Change'].iloc[i] > 0 and in_pattern:
        pattern_end.append(i)
        in_pattern = False

# 输出匹配到的模式数量
pattern_count = len(pattern_start)
print(f'Pattern识别结果的数量: {pattern_count}')

Pattern识别结果的数量: 1226


In [77]:
import pandas as pd
import numpy as np

# 读取数据
file_path = './nasdaq/csv/GOOD.csv'
df = pd.read_csv(file_path)

# 转换日期为 datetime 格式
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

# 定义事件 A: Close 值下降, 事件 B: Close 值上升
df['Close_Change'] = df['Close'].diff()

# 初始化状态机识别
pattern_start = []
pattern_end = []

for i in range(1, len(df)):
    # 识别事件A: Close值下降
    if df['Close_Change'].iloc[i] < 0:
        start_index = i
        # skip-till-any-match: 允许跳过多个事件A，直到找到事件B
        for j in range(i + 1, len(df)):
            if df['Close_Change'].iloc[j] > 0:  # 识别事件B: Close值上升
                pattern_start.append(start_index)
                pattern_end.append(j)
                break

# 输出匹配到的模式数量
pattern_count = len(pattern_start)
print(f'Pattern识别结果的数量: {pattern_count}')

# 计算每个模式的时间跨度
longest_duration = pd.Timedelta(0)
longest_start = None
longest_end = None

for start, end in zip(pattern_start, pattern_end):
    duration = df['Date'].iloc[end] - df['Date'].iloc[start-1]
    if duration > longest_duration:
        longest_duration = duration
        longest_start = df['Date'].iloc[start-1]
        longest_end = df['Date'].iloc[end]

# 输出最长的时间区间
if longest_start is not None and longest_end is not None:
    print(f'最长的时间区间是从 {longest_start} 到 {longest_end}, 跨度为 {longest_duration}')

Pattern识别结果的数量: 2324
最长的时间区间是从 2004-03-29 00:00:00 到 2004-04-16 00:00:00, 跨度为 18 days 00:00:00


In [25]:
len(df)

4301

In [3]:
import os
import pandas as pd

# 定义CSV文件所在目录
directory = './nasdaq/csv'

# 初始化一个空的DataFrame用于存放所有文件的数据
all_data = pd.DataFrame()

# 遍历目录下的所有文件
size = 100
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        size -= 1
        if size < 0:
            break
        # 构造文件路径
        file_path = os.path.join(directory, filename)
        
        # 读取CSV文件
        df = pd.read_csv(file_path)
        
        # 提取文件名（去掉扩展名 .csv）
        name = os.path.splitext(filename)[0]
        
        # 添加新的列 'Name'，列值为文件名
        df['Name'] = name
        
        # 转换 'Date' 列为 datetime 类型
        df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
        
        # 将数据拼接到 all_data 中
        all_data = pd.concat([all_data, df], ignore_index=True)

        
# 按照 'Date' 和 'Name' 进行排序
all_data = all_data.sort_values(by=['Date', 'Name'])

# 输出合并后的数据
print(all_data.head())

             Date     Low    Open  Volume    High   Close  Adjusted Close  \
453281 1973-05-03  3.6250  3.6250  2000.0  3.8125  3.6250        2.037348   
453282 1973-05-04  3.6250  3.6250  1600.0  3.8125  3.6250        2.037348   
453283 1973-05-07  3.6250  3.6250  1600.0  3.8125  3.6250        2.037348   
453284 1973-05-08  3.6875  3.6875  1600.0  3.8750  3.6875        2.072475   
453285 1973-05-09  3.6875  3.6875  6000.0  3.8750  3.6875        2.072475   

        Name  
453281  ALCO  
453282  ALCO  
453283  ALCO  
453284  ALCO  
453285  ALCO  


In [13]:
len(all_data)

589134

In [1]:
import os
import pandas as pd

# 定义CSV文件所在目录
directory = './nasdaq/csv'

# 初始化一个空的列表用于存放结果
results = []

# 遍历目录下的所有文件
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # 构造文件路径
        file_path = os.path.join(directory, filename)
        
        # 读取CSV文件，处理错误行
        try:
            df = pd.read_csv(file_path, on_bad_lines='skip')
            
            # 转换 'Date' 列为 datetime 类型
            df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
            
            # 获取第一个和最后一个时间戳
            first_timestamp = df['Date'].min()
            last_timestamp = df['Date'].max()
            
            # 将结果添加到 results 列表中
            results.append({
                'File Name': os.path.splitext(filename)[0],
                'First Timestamp': first_timestamp,
                'Last Timestamp': last_timestamp
            })
        except Exception as e:
            print(f"Error processing file {filename}: {e}")

# 使用 pd.DataFrame 将结果转换为 DataFrame
timestamps = pd.DataFrame(results)

# 输出结果
print(timestamps)

KeyboardInterrupt: 

In [26]:
# 获取最大的 First Timestamp
max_first_timestamp = timestamps['First Timestamp'].max()

# 输出最大的 First Timestamp
print(f"最大的 First Timestamp 是: {max_first_timestamp}")

NameError: name 'timestamps' is not defined

In [19]:
# 统计每个 First Timestamp 的数量
first_timestamp_counts = timestamps['First Timestamp'].value_counts()

# 输出每个 First Timestamp 的数量
print(first_timestamp_counts)

1980-03-17    56
1994-04-04     7
1973-02-21     6
1999-01-04     6
2003-10-07     5
              ..
1998-05-21     1
1993-11-01     1
1992-07-01     1
2002-09-11     1
2010-03-01     1
Name: First Timestamp, Length: 1277, dtype: int64


In [3]:
import os
import pandas as pd

# 定义CSV文件所在目录
directory = './nasdaq/csv'

# 定义过滤的日期
filter_date = pd.to_datetime('1980-03-17')

# 初始化一个空的DataFrame用于存放结果
filtered_data = pd.DataFrame()

# 遍历目录下的所有文件
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # 构造文件路径
        file_path = os.path.join(directory, filename)
        
        # 读取CSV文件，并跳过有问题的行
        df = pd.read_csv(file_path, on_bad_lines='skip')
        
        # 转换 'Date' 列为 datetime 类型
        df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y', errors='coerce')
        
        # 检查第一个时间戳是否早于1980-03-17
        first_timestamp = df['Date'].min()
        if first_timestamp < filter_date:
            # 保留1980-03-17之后的数据
            df_filtered = df[df['Date'] >= filter_date].copy()  # 使用 .copy() 防止视图问题
            
            # 添加一个新列 'Name' 以标识文件来源
            df_filtered.loc[:, 'Name'] = os.path.splitext(filename)[0]  # 使用 .loc 进行赋值
            
            # 将结果收集到 filtered_data 中
            filtered_data = pd.concat([filtered_data, df_filtered], ignore_index=True)

# 对结果先按 'Date' 排序，再按 'Name' 排序
filtered_data = filtered_data.sort_values(by=['Date', 'Name'])

# 输出结果
print(filtered_data)

             Date         Low        Open    Volume        High       Close  \
10778  1980-03-17    7.625000    7.875000    8800.0    8.125000    7.625000   
172448 1980-03-17    5.062500    5.062500   17600.0    5.250000    5.062500   
140114 1980-03-17    1.170000    1.170000     694.0    1.278000    1.170000   
64668  1980-03-17    0.320988    0.000000   10125.0    0.345679    0.320988   
161670 1980-03-17    5.500000    5.562500  210600.0    5.625000    5.625000   
...           ...         ...         ...       ...         ...         ...   
32333  2022-12-12    7.465000    7.484800    1415.0    7.488700    7.480000   
64667  2022-12-12  172.910004  174.110001  748248.0  174.445007  174.020004   
43111  2022-12-12   23.520100   24.170000   12485.0   24.170000   23.770100   
86223  2022-12-12   35.340000   35.529999  943656.0   35.840000   35.669998   
53889  2022-12-12         NaN         NaN       NaN         NaN         NaN   

        Adjusted Close   Name  
10778         4.285

In [4]:
import pandas as pd
import time

# 开始时间
start_time = time.time()

# 假设 filtered_data 是已经处理过的数据
# filtered_data 应包含 'Date', 'Name', 'Open', 'Close', 'High' 等列

# 确保 Date 列为 datetime 类型
filtered_data['Date'] = pd.to_datetime(filtered_data['Date'])

# 将数据按 Name 和 Date 排序
filtered_data = filtered_data.sort_values(by=['Name', 'Date'])

# 初始化用于存储模式匹配的结果
pattern_results = []

# 遍历每个股票的组
for name, group in filtered_data.groupby('Name'):
    group['Up'] = group['Close'] > group['Open']  # 判断是否上涨
    group['Up_Streak'] = group['Up'].astype(int).rolling(window=5, min_periods=5).sum()  # 计算连续上涨次数

    # 过滤出连续上涨且持续3天以上的日期
    valid_streaks = group[(group['Up_Streak'] >= 5) & (group['Up'])]
    
    if not valid_streaks.empty:
        for idx in valid_streaks.index:
            # 确定时间窗口的开始日期
            start_date = valid_streaks.at[idx, 'Date']
            end_date = start_date + pd.Timedelta(days=30)  # 计算30天后的结束日期
            
            # 检查其他股票在此日期之后的30天内是否有 Close > High
            subsequent_dates = filtered_data[(filtered_data['Date'] > start_date) & 
                                              (filtered_data['Date'] <= end_date) & 
                                              (filtered_data['Name'] != name)]
            if not subsequent_dates.empty:
                b_condition_met = subsequent_dates[subsequent_dates['Close'] > subsequent_dates['High']]
                if not b_condition_met.empty:
                    pattern_results.append({
                        'Stock Name': name,
                        'Start Date': start_date,
                        'Matched Stock': b_condition_met['Name'].unique(),
                        'Matched Date': b_condition_met['Date'].unique()
                    })

# 输出匹配的结果
results_df = pd.DataFrame(pattern_results)
print(results_df)

# 计算最长时间跨度
if not results_df.empty:
    max_span = results_df['Start Date'].max() - results_df['Start Date'].min()
    print("最长时间跨度：", max_span)
else:
    print("未找到符合条件的模式。")

# 结束时间
end_time = time.time()
print("代码运行时间：{:.2f}秒".format(end_time - start_time))

    Stock Name Start Date Matched Stock  \
0         KLIC 1980-03-21        [DIOD]   
1         KLIC 1980-03-24        [DIOD]   
2         KLIC 1980-03-25        [DIOD]   
3         KLIC 1980-03-26        [DIOD]   
4         KLIC 1980-03-27        [DIOD]   
..         ...        ...           ...   
141        WDC 1980-04-25        [DIOD]   
142        WDC 1980-04-28        [DIOD]   
143        WDC 1980-04-29        [DIOD]   
144        WDC 1980-04-30        [DIOD]   
145        WDC 1980-05-01        [DIOD]   

                                          Matched Date  
0                      [1980-04-15T00:00:00.000000000]  
1                      [1980-04-15T00:00:00.000000000]  
2                      [1980-04-15T00:00:00.000000000]  
3                      [1980-04-15T00:00:00.000000000]  
4                      [1980-04-15T00:00:00.000000000]  
..                                                 ...  
141  [1980-04-29T00:00:00.000000000, 1980-05-02T00:...  
142  [1980-04-29T00:00:00.0

In [30]:
len(results_df)

146

In [5]:
# 按照每只股票进行分组，计算各个股票的闪崩次数
def detect_flash_crash(group):
    # 定义条件：价格下跌超过5%
    group['Flash_Crash'] = ((group['High'] - group['Low']) / group['High']) >= 0.05
    
    # 可以根据需要添加其他条件，比如成交量异常增高等
    # 比如说当天成交量大于前一天的1.5倍
    group['Volume_Increase'] = group['Volume'].pct_change() > 0.5
    
    # 仅保留满足上述两个条件的行
    group['Flash_Crash'] = group['Flash_Crash'] & group['Volume_Increase']
    
    # 统计闪崩次数
    return group['Flash_Crash'].sum()

# 应用函数，按股票名称分组
flash_crash_counts = filtered_data.groupby('Name').apply(detect_flash_crash)

# 查看统计结果
print(flash_crash_counts)

Name
ALCO      841
ALOG      413
APOG      751
DIOD     1200
GT        428
HELE      910
KLIC     1102
MAT       525
MSEX      303
OTTR      174
PHI       401
SGC       496
SHLM      388
TRNS     2157
TSRI     1704
TXN       286
VLGEA     727
WDC      1048
WSCI     1354
dtype: int64


In [6]:
import pandas as pd
from sklearn.linear_model import LinearRegression


# 假设 filtered_data 是你的输入数据
# filtered_data 应包含 'Date', 'Close', 'Name' 等字段

def detect_significant_upward_trend(stock_data, min_days=5, max_days=10, slope_threshold=2):
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])
    stock_data.sort_values(by='Date', inplace=True)
    results = []

    unique_stocks = stock_data['Name'].unique()

    for stock in unique_stocks:
        current_stock_data = stock_data[stock_data['Name'] == stock]
        close_prices = current_stock_data['Close'].values

        # 提取30天内的所有组合
        if len(close_prices) >= max_days:
            for i in range(len(close_prices) - max_days + 1):
                for n in range(min_days, max_days + 1):
                    for comb in combinations(range(i, i + max_days), n):
                        # 获取组合对应的收盘价
                        selected_prices = close_prices[list(comb)]

                        # 线性回归计算斜率
                        X = np.arange(len(selected_prices)).reshape(-1, 1)
                        model = LinearRegression()
                        model.fit(X, selected_prices)
                        slope = model.coef_[0]  # 获取斜率

                        # 如果斜率大于2，认为是显著上升
                        if slope > slope_threshold:
                            results.append((stock, selected_prices, slope))

    return results

flash_crash_data = filtered_data[filtered_data['Name'] == "ALCO"]

# 开始时间
start_time = time.time()

# 运行函数
result = detect_significant_upward_trend(filtered_data)

# 结束时间
end_time = time.time()
print("代码运行时间：{:.2f}秒".format(end_time - start_time))

# 打印结果
for stock, prices, slope in result:
    print(f'Stock: {stock}, Prices: {prices}, Slope: {slope}')

ValueError: Input y contains NaN.

In [6]:
import numpy as np
import pandas as pd
import time
from itertools import combinations

def calculate_slope(dates, prices):
    """
    计算给定日期和价格的斜率
    使用最小二乘法公式计算斜率：
    slope = (N*Σxy - Σx*Σy) / (N*Σx^2 - (Σx)^2)
    """
    N = len(dates)
    sum_x = np.sum(dates)
    sum_y = np.sum(prices)
    sum_xy = np.sum(dates * prices)
    sum_x_squared = np.sum(dates ** 2)

    numerator = N * sum_xy - sum_x * sum_y
    denominator = N * sum_x_squared - sum_x ** 2

    return numerator / denominator if denominator != 0 else 0

def detect_flash_crash(stock_data, min_days=5, max_days=10, slope_threshold=0.1, crash_threshold=0.5):
    """
    检测股票数据中的闪崩事件，只遍历一次数据。
    """
    stock_data = stock_data.copy()
    stock_data['Date'] = pd.to_datetime(stock_data['Date'], format='%Y-%m-%d')
    stock_data.sort_values(by='Date', inplace=True)
    stock_data['Date'] = stock_data['Date'].map(pd.Timestamp.toordinal)

    result_list = []
    unique_stocks = stock_data['Name'].unique()

    for stock in unique_stocks:
        current_stock_data = stock_data[stock_data['Name'] == stock]
        close_prices = current_stock_data['Close'].values
        low_prices = current_stock_data['Low'].values
        high_prices = current_stock_data['High'].values
        dates = current_stock_data['Date'].values

        # 遍历每一个可能的崩溃事件
        for j in range(len(close_prices)):
            price_drop = high_prices[j] - low_prices[j]
            if price_drop >= crash_threshold:
                crash_date = dates[j]

                # 查找崩溃日期前10天内的数据
                flash_candidates = dates[max(0, j - 9):j]
                candidate_prices = close_prices[max(0, j - 9):j]
                
                if len(flash_candidates) < 5:
                    continue

                # 判断是否满足min和max条件
                if min(candidate_prices) < low_prices[j] and max(candidate_prices) > high_prices[j]:
                    # 满足min和max条件后生成组合
                    for n in range(min_days, min(max_days, len(flash_candidates)) + 1):
                        for comb in combinations(range(len(flash_candidates)), n):
                            selected_dates = flash_candidates[list(comb)]
                            selected_prices = candidate_prices[list(comb)]

                            # 计算斜率
                            slope = calculate_slope(selected_dates, selected_prices)
                            if slope > slope_threshold:
                                begin_date = pd.Timestamp.fromordinal(selected_dates[0]).strftime('%Y-%m-%d')
                                end_date = pd.Timestamp.fromordinal(crash_date).strftime('%Y-%m-%d')

                                flash_stage = [pd.Timestamp.fromordinal(ts).strftime('%Y-%m-%d') 
                                               for ts in selected_dates]

                                result_list.append({
                                    "begin_date": begin_date,
                                    "end_date": end_date,
                                    "flash_stage": ";".join(flash_stage),
                                    "crash_stage": pd.Timestamp.fromordinal(crash_date).strftime('%Y-%m-%d')
                                })

    return result_list

# 假设 filtered_data 是输入数据
flash_crash_data = filtered_data[filtered_data['Name'] == "ALCO"]

start_time = time.time()
result = detect_flash_crash(flash_crash_data)
end_time = time.time()

print(f"代码运行时间：{end_time - start_time:.2f}秒")
print(f"匹配的结果数量: {len(result)}")

# 将结果转换为DataFrame并保存为CSV文件
df = pd.DataFrame(result)
df.to_csv("result.csv", index=False)

print("结果已保存到 result.csv")

代码运行时间：6.15秒
匹配的结果数量: 114333
结果已保存到 result.csv


In [14]:
len(df.unique())

AttributeError: 'DataFrame' object has no attribute 'unique'

In [54]:
flash_crash_data.head(5)

,Date,Low,Open,Volume,High,Close,Adjusted Close,Name
10778,1980-03-17,7.625,7.875,8800.0,8.125,7.625,4.285454,ALCO
10779,1980-03-18,7.625,7.625,5200.0,8.125,7.625,4.285454,ALCO
10780,1980-03-19,7.500,7.625,12400.0,8.000,7.500,4.215203,ALCO
10781,1980-03-20,7.375,7.500,1600.0,7.875,7.375,4.144949,ALCO
10782,1980-03-21,7.375,7.375,0.0,7.375,7.375,4.144949,ALCO


In [46]:
# Assuming flash_crash_data is a pandas DataFrame
flash_crash_data.to_csv('ALCO.csv', index=False)  # Export to CSV without the index column

In [47]:
flash_crash_data.head(5)

,Date,Low,Open,Volume,High,Close,Adjusted Close,Name
10778,1980-03-17,7.625,7.875,8800.0,8.125,7.625,4.285454,ALCO
10779,1980-03-18,7.625,7.625,5200.0,8.125,7.625,4.285454,ALCO
10780,1980-03-19,7.500,7.625,12400.0,8.000,7.500,4.215203,ALCO
10781,1980-03-20,7.375,7.500,1600.0,7.875,7.375,4.144949,ALCO
10782,1980-03-21,7.375,7.375,0.0,7.375,7.375,4.144949,ALCO


In [27]:
print(result[0])

{'begin_date': '1985-08-26', 'end_date': '1985-09-04', 'flash_stage': '1985-08-26;1985-08-27;1985-08-28;1985-08-29;1985-09-03', 'crash_stage': '1985-09-04'}


In [35]:
len(result_df)

4268

原始 result_df 行数: 866
原始 latency_df 行数: 4794
重叠的部分行数（交集）: 100
只在 result_df 中的部分行数: 766
只在 latency_df 中的部分行数: 4694
合计行数: 5560
预期行数（result_df + latency_df - overlap_df）: 5560


2022-04-12;2022-04-13;2022-04-14;2022-04-18;2022-04-19;2022-04-20;2022-04-21
2022-10-10;2022-10-11;2022-10-12;2022-10-13;2022-10-17


事件检测结果：满足事件条件


True

[724879 724880 724881 724882 724883]
0.1625
[724879 724880 724881 724882 724887]
0.15206185567010308
[724879 724880 724881 724882 724888]
0.13
[724879 724880 724881 724883 724887]
0.1390625
[724879 724880 724881 724883 724888]
0.12401574803149606
[724879 724880 724881 724887 724888]
0.13214285714285715
[724879 724880 724882 724883 724887]
0.1559278350515464
[724879 724880 724882 724883 724888]
0.13795731707317074
[724879 724880 724882 724887 724888]
0.14296407185628743
[724879 724880 724883 724887 724888]
0.15529141104294478
[724879 724881 724882 724883 724887]
0.11754261363636363
[724879 724881 724882 724883 724888]
0.103429203539823
[724879 724881 724882 724887 724888]
0.1053921568627451
[724879 724881 724883 724887 724888]
0.1176097972972973
[724879 724882 724883 724887 724888]
0.13138686131386862
[724880 724881 724882 724883 724887]
0.1172945205479452
[724880 724881 724882 724883 724888]
0.10212628865979381
[724880 724881 724882 724887 724888]
0.10432330827067669
[724880 724881 724

In [57]:
unique_to_result_df.head(1)

,begin_date,end_date,flash_stage,crash_stage
0,1985-08-26,1985-09-04,1985-08-26;1985-08-27;1985-08-28;1985-08-29;19...,1985-09-04


In [48]:
# 找出持续时间最短的结果
shortest_result = min(result, key=lambda x: (x[3][1] - x[3][0]).days)

print("持续时间最短的结果为:", shortest_result)

持续时间最短的结果为: ('ALCO', array([22.25, 22.25, 22.25, 22.25, 23.25]), 0.16981132075471697, (Timestamp('1986-02-25 00:00:00'), Timestamp('1986-03-04 00:00:00')))


In [49]:
# 设定日期范围
start_date = pd.Timestamp('1986-02-25')
end_date = pd.Timestamp('1986-03-04')

# 筛选出日期范围内的数据
flash_crash_data_succeed_demo = flash_crash_data[
    (flash_crash_data['Date'] >= start_date) & (flash_crash_data['Date'] <= end_date)
]

# 保存为 CSV 文件
flash_crash_data_succeed_demo.to_csv("flash_crash_data_succeed_demo.csv", index=False)


In [37]:
import pandas as pd
import numpy as np
from itertools import combinations
import time

def calculate_slope(dates, prices):
    """
    计算给定日期和价格的斜率
    使用最小二乘法公式计算斜率：slope = (N*Σxy - Σx*Σy) / (N*Σx^2 - (Σx)^2)
    """
    N = len(dates)
    sum_x = np.sum(dates)
    sum_y = np.sum(prices)
    sum_xy = np.sum(dates * prices)
    sum_x_squared = np.sum(dates ** 2)

    numerator = N * sum_xy - sum_x * sum_y
    denominator = N * sum_x_squared - sum_x ** 2

    return numerator / denominator if denominator != 0 else 0

def detect_flash_crash(stock_data, min_days=5, max_days=10, slope_threshold=2, crash_threshold=-5):
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])
    stock_data.sort_values(by='Date', inplace=True)
    stock_data['Date'] = stock_data['Date'].map(pd.Timestamp.toordinal)  # 将日期转换为整数

    results = []
    unique_stocks = stock_data['Name'].unique()

    for stock in unique_stocks:
        current_stock_data = stock_data[stock_data['Name'] == stock]
        close_prices = current_stock_data['Close'].values
        dates = current_stock_data['Date'].values

        # 提取30天内的所有组合
        if len(close_prices) >= max_days:
            for i in range(len(close_prices) - max_days + 1):
                for n in range(min_days, max_days + 1):
                    for comb in combinations(range(i, i + max_days), n):
                        # 获取组合对应的日期和收盘价
                        selected_dates = dates[list(comb)]
                        selected_prices = close_prices[list(comb)]

                        # 计算斜率，检测上升趋势
                        slope = calculate_slope(selected_dates, selected_prices)

                        # 如果斜率大于2，认为是显著上升
                        if slope > slope_threshold:
                            # 检查上升趋势后的10天内是否出现闪崩
                            crash_detected = False
                            if i + max_days < len(close_prices) - 10:
                                future_prices = close_prices[i + max_days:i + max_days + 10]
                                # 计算未来10天内的价格变化百分比
                                price_drop = (future_prices[0] - future_prices[-1]) / future_prices[0] * 100
                                if price_drop < crash_threshold:  # 认为价格下跌超过5%为闪崩
                                    crash_detected = True

                            if crash_detected:
                                results.append((stock, selected_prices, slope, price_drop))

    return results

# 测试数据（根据你的需求替换为实际的 filtered_data）
flash_crash_data = filtered_data[filtered_data['Name'] == "ALCO"]

# 开始时间
start_time = time.time()

# 运行函数
result = detect_flash_crash(flash_crash_data)

# 结束时间
end_time = time.time()
print("代码运行时间：{:.2f}秒".format(end_time - start_time))

# 打印结果
for stock, prices, slope, price_drop in result:
    print(f'Stock: {stock}, Prices: {prices}, Slope: {slope}, Flash Crash Drop: {price_drop}%')

/var/folders/n6/_bh3pvzd13s0pgv_hdy0np780000gn/T/ipykernel_40337/404727076.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data['Date'] = pd.to_datetime(stock_data['Date'])
/var/folders/n6/_bh3pvzd13s0pgv_hdy0np780000gn/T/ipykernel_40337/404727076.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data.sort_values(by='Date', inplace=True)
/var/folders/n6/_bh3pvzd13s0pgv_hdy0np780000gn/T/ipykernel_40337/404727076.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

KeyboardInterrupt: 

In [169]:
from itertools import combinations
import numpy as np
import pandas as pd
import time

def calculate_slope(dates, prices):
    """
    计算给定日期和价格的斜率
    使用最小二乘法公式计算斜率：
    slope = (N*Σxy - Σx*Σy) / (N*Σx^2 - (Σx)^2)
    """
    dates = np.array(dates)  # 转换为 NumPy 数组
    prices = np.array(prices)  # 转换为 NumPy 数组
    
    N = len(dates)
    sum_x = np.sum(dates)
    sum_y = np.sum(prices)
    sum_xy = np.sum(dates * prices)
    sum_x_squared = np.sum(dates ** 2)

    numerator = N * sum_xy - sum_x * sum_y
    denominator = N * sum_x_squared - sum_x ** 2

    return numerator / denominator if denominator != 0 else 0

def get_recent_days_data(dates, prices, crash_date, max_days):
    """
    从崩溃日期向前查找不超过 max_days 天的数据
    """
    flash_candidates = []
    candidate_prices = []
    
    # 从崩溃日期往前查找不超过 max_days 的数据
    for i in range(len(dates) - 1, -1, -1):
        date_diff = (pd.Timestamp(crash_date) - pd.Timestamp(dates[i])).days
        if date_diff > 0 and date_diff < max_days:
            flash_candidates.append(dates[i])
            candidate_prices.append(prices[i])
        elif date_diff >= max_days:
            break  # 超出 max_days 时停止查找

    # 返回按时间顺序排列的数据
    return flash_candidates[::-1], candidate_prices[::-1]

def detect_flash_crash(stock_data, min_days=5, max_days=10, slope_threshold=0.1, crash_threshold=0.5):
    stock_data = stock_data.copy()
    stock_data['Date'] = pd.to_datetime(stock_data['Date'], format='%Y-%m-%d')
    stock_data.sort_values(by='Date', inplace=True)

    result_list = []
    unique_stocks = stock_data['Name'].unique()

    for stock in unique_stocks:
        current_stock_data = stock_data[stock_data['Name'] == stock]
        close_prices = current_stock_data['Close'].values
        low_prices = current_stock_data['Low'].values
        high_prices = current_stock_data['High'].values
        dates = current_stock_data['Date'].values

        # 遍历每一个可能的崩溃事件
        for j in range(len(close_prices)):
            price_drop = high_prices[j] - low_prices[j]
            if price_drop >= crash_threshold:
                crash_date = dates[j]
                
                # 查找崩溃日期前 max_days 天内的数据
                flash_candidates, candidate_prices = get_recent_days_data(dates[:j], close_prices[:j], crash_date, max_days)
                
                if len(flash_candidates) < min_days:
                    continue
                
                # 衍生约束
                if max(candidate_prices) <= low_prices[j] and min(candidate_prices) >= high_prices[j]:
                    continue

                # 满足 min 和 max 条件后生成唯一组合
                for n in range(min_days, min(max_days, len(flash_candidates) + 1)):
                    unique_combinations = set(combinations(range(len(flash_candidates)), n))
                    for comb in unique_combinations:
                        selected_dates = [pd.Timestamp(flash_candidates[i]).toordinal() for i in comb]
                        selected_prices = [candidate_prices[i] for i in comb]
                        if min(selected_prices) < low_prices[j] or max(selected_prices) > high_prices[j]:
                            continue 

                        # 计算斜率
                        slope = calculate_slope(selected_dates, selected_prices)
                        if slope >= slope_threshold:
                            begin_date = pd.Timestamp(flash_candidates[comb[0]]).strftime('%Y-%m-%d')

                            end_date = pd.Timestamp(crash_date).strftime('%Y-%m-%d')

                            # flash_stage 仅包括 selected_dates 转换后的日期
                            flash_stage = [pd.Timestamp(flash_candidates[i]).strftime('%Y-%m-%d') for i in comb]

                            result_dict = {
                                "begin_date": begin_date,
                                "end_date": end_date,
                                "flash_stage": ";".join(flash_stage),
                                "crash_stage": pd.Timestamp(crash_date).strftime('%Y-%m-%d')
                            }
                            if result_dict not in result_list:
                                result_list.append(result_dict)
                    
    return result_list

# 假设 filtered_data 是输入数据
flash_crash_data = pd.read_csv('ALCO.csv')
flash_crash_data.head(5)

start_time = time.time()
result = detect_flash_crash(flash_crash_data)
end_time = time.time()

print(f"代码运行时间：{end_time - start_time:.2f}秒")
print(f"匹配的结果数量: {len(result)}")

# 将结果转换为 DataFrame 并保存为 CSV 文件
df = pd.DataFrame(result)
df.to_csv("result.csv", index=False)

print("结果已保存到 result.csv")

代码运行时间：1.02秒
匹配的结果数量: 1281
结果已保存到 result.csv


In [162]:
result_df = pd.DataFrame(result)
# result_df = pd.read_csv('lazy.csv')
latency_df = pd.read_csv('latency_handle.csv')
# 确保数据唯一性
result_df = result_df.drop_duplicates().reset_index(drop=True)
latency_df = latency_df.drop_duplicates().reset_index(drop=True)

# 计算原始行数
print("原始 result_df 行数:", len(result_df))
print("原始 latency_df 行数:", len(latency_df))

# 找到重叠部分（交集）
overlap_df = pd.merge(result_df, latency_df, on=['begin_date', 'end_date', 'flash_stage', 'crash_stage'], how='inner')
print("重叠的部分行数（交集）:", len(overlap_df))

# 找到不重叠部分（差集）
unique_to_result_df = pd.concat([result_df, overlap_df]).drop_duplicates(keep=False).reset_index(drop=True)
unique_to_latency_df = pd.concat([latency_df, overlap_df]).drop_duplicates(keep=False).reset_index(drop=True)

# 输出各部分的行数
print("只在 result_df 中的部分行数:", len(unique_to_result_df))
print("只在 latency_df 中的部分行数:", len(unique_to_latency_df))

# 确认总行数
total_rows = len(overlap_df) + len(unique_to_result_df) + len(unique_to_latency_df)
print("合计行数:", total_rows)
print("预期行数（result_df + latency_df - overlap_df）:", len(result_df) + len(latency_df) - len(overlap_df))

原始 result_df 行数: 394
原始 latency_df 行数: 1282
重叠的部分行数（交集）: 394
只在 result_df 中的部分行数: 0
只在 latency_df 中的部分行数: 888
合计行数: 1282
预期行数（result_df + latency_df - overlap_df）: 1282


In [158]:
# 输出 unique_to_latency_df 的最后一条数据的 'flash_stage' 列
print(unique_to_latency_df.tail(1)['flash_stage'].values[0])

1996-03-29;1996-04-01;1996-04-02;1996-04-03;1996-04-04


In [159]:
unique_to_latency_df.tail(1)

,begin_date,end_date,flash_stage,crash_stage
0,1996-03-29,1996-04-08,1996-03-29;1996-04-01;1996-04-02;1996-04-03;19...,1996-04-08


In [160]:
import pandas as pd
import numpy as np
from datetime import datetime

class FlashCrashTest:
    def __init__(self, data, result_df, slope_threshold=0.1):
        """
        初始化测试类
        :param data: 股票数据的 DataFrame，需包含 Date, Name, Close 列
        :param result_df: 现有事件记录的 DataFrame，用于避免重复检查
        :param slope_threshold: 用于检测上升趋势的斜率阈值
        """
        self.data = data
        self.result_df = result_df
        self.data['Date'] = pd.to_datetime(self.data['Date'])
        self.result_df['begin_date'] = pd.to_datetime(self.result_df['begin_date'])
        self.result_df['end_date'] = pd.to_datetime(self.result_df['end_date'])
        self.slope_threshold = slope_threshold

    def calculate_slope(self, dates, prices):
        """
        计算斜率
        """
        dates = np.array(dates)
        prices = np.array(prices)
        
        N = len(dates)
        sum_x = np.sum(dates)
        sum_y = np.sum(prices)
        sum_xy = np.sum(dates * prices)
        sum_x_squared = np.sum(dates ** 2)

        numerator = N * sum_xy - sum_x * sum_y
        denominator = N * sum_x_squared - sum_x ** 2

        return numerator / denominator if denominator != 0 else 0

    def check_event(self, flash_dates_str, crash_date_str):
        """
        检查给定日期中是否发生事件
        :param flash_dates_str: Flash阶段的日期字符串，格式为 "YYYY-MM-DD;YYYY-MM-DD;..."
        :param crash_date_str: Crash阶段的日期字符串，格式为 "YYYY-MM-DD"
        :return: bool，是否发生事件
        """
        # 解析日期
        flash_dates = [datetime.strptime(date.strip(), '%Y-%m-%d') for date in flash_dates_str.split(';')]
        crash_date = datetime.strptime(crash_date_str.strip(), '%Y-%m-%d')
        begin_date = flash_dates[0]

        # 首先检查是否已在 result_df 中
        existing_event = self.result_df[
            (self.result_df['begin_date'] == begin_date) & (self.result_df['end_date'] == crash_date)&
            (self.result_df['flash_stage'] == ";".join([date.strftime('%Y-%m-%d') for date in flash_dates]))
        ]
        
        if not existing_event.empty:
            print("事件已存在于 result_df 中")
            return True
        else:
            print("事件不存在于 result_df 中")
        # 筛选 ALCO 股票数据
        stock_data = self.data[self.data['Name'] == 'ALCO']

        # 获取 flash 和 crash 阶段的收盘价数据
        flash_data = stock_data[stock_data['Date'].isin(flash_dates)]
        crash_data = stock_data[stock_data['Date'] == crash_date]
        print(flash_data)
        print(crash_data)

        if flash_data.empty or crash_data.empty:
            print("指定的 flash 或 crash 日期数据缺失")
            return False

        # 计算 flash 阶段的斜率
        flash_ordinal_dates = flash_data['Date'].map(datetime.toordinal).values
        flash_prices = flash_data['Close'].values
        slope = self.calculate_slope(flash_ordinal_dates, flash_prices)
        print(flash_prices)

        # 检查是否满足上升斜率和 crash 阶段的价格低于 flash 阶段的最高值
        if slope >= self.slope_threshold and crash_data['High'].values[0] >= max(flash_prices) and crash_data['Low'].values[0] <= min(flash_prices):
            print("事件检测结果：满足事件条件")
            return True
        else:
            print("事件检测结果：不满足事件条件")
            return False

# 读取数据
data = pd.read_csv('ALCO.csv')

# 示例输入1987-03-23,1987-04-01,,

tester = FlashCrashTest(data, result_df)
flash_dates = "1996-03-29;1996-04-01;1996-04-02;1996-04-03;1996-04-04"
crash_date = "1996-04-08"
tester.check_event(flash_dates, crash_date)

事件不存在于 result_df 中
           Date      Low   Open   Volume   High  Close  Adjusted Close  Name
4055 1996-03-29  22.5000  23.00    800.0  23.00  22.50       13.490532  ALCO
4056 1996-04-01  22.5000  22.50   5000.0  23.25  22.50       13.490532  ALCO
4057 1996-04-02  21.9375  22.25  23700.0  23.25  22.75       13.640431  ALCO
4058 1996-04-03  23.5000  23.50   1600.0  23.50  23.50       14.090112  ALCO
4059 1996-04-04  23.5000  23.50      0.0  23.50  23.50       14.090112  ALCO
           Date    Low   Open  Volume  High  Close  Adjusted Close  Name
4060 1996-04-08  22.25  22.25  7400.0  23.5   23.5       14.090112  ALCO
[22.5  22.5  22.75 23.5  23.5 ]
事件检测结果：满足事件条件


True

In [14]:
flash_crash_data.head(5)


,Date,Low,Open,Volume,High,Close,Adjusted Close,Name
0,1980-03-17,7.625,7.875,8800.0,8.125,7.625,4.285454,ALCO
1,1980-03-18,7.625,7.625,5200.0,8.125,7.625,4.285454,ALCO
2,1980-03-19,7.500,7.625,12400.0,8.000,7.500,4.215203,ALCO
3,1980-03-20,7.375,7.500,1600.0,7.875,7.375,4.144949,ALCO
4,1980-03-21,7.375,7.375,0.0,7.375,7.375,4.144949,ALCO


In [4]:
from collections import Counter

# 将字典列表转化为不可变的格式，以便检查重复项
result_tuples = [tuple(sorted(item.items())) for item in result]
# 使用 Counter 统计出现次数
duplicates = [item for item, count in Counter(result_tuples).items() if count > 1]

# 输出结果
if duplicates:
    print("Result 中存在重复项：")
    for duplicate in duplicates:
        print(dict(duplicate))  # 将元组还原为字典格式输出
else:
    print("Result 中没有重复项")

Result 中没有重复项


In [6]:
result[0]

{'begin_date': '1981-06-09',
 'end_date': '1981-06-22',
 'flash_stage': '1981-06-09;1981-06-10;1981-06-11;1981-06-12;1981-06-15',
 'crash_stage': '1981-06-22'}